In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", None)
import tkinter as tk
from tkinter import filedialog
import plotly.express as px

In [ ]:
def select_sheet():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    df = pd.read_excel(file_path, None)
    list_dict = {}
    for k, v in enumerate(list(df.keys())):
        print(k+1, v)
        list_dict.setdefault(k+1, v)
    sheet_name = eval(input("Please enter the sheet No.:"))
    sheet_name = list_dict[sheet_name]
    return df[sheet_name]

In [ ]:
df = select_sheet()

In [ ]:
df = df[(df["狀態"].str.contains("出") & (df["負責業務"] == "許凱智"))]

In [ ]:
test = df.groupby(["Group"])[["數量"]].sum().reset_index()

In [ ]:
px.bar(test, x="Group", y="數量", log_y=True).update_xaxes(categoryorder="total descending")

## 20210223俄羅斯元件排名

In [ ]:
df = pd.read_clipboard()

In [ ]:
df.columns = ["Place in 2018", "Place in 2012", "Distributor", "Founded", "Sales volume USD bln, 2018", "Affiliates/offices/representatives", "Employees"]

In [ ]:
df["Place in 2012"] = df["Place in 2012"].astype("str")

In [ ]:
df["Place in 2012"] = df["Place in 2012"].str.split(".", expand=True)[0]

In [ ]:
df["Place in 2018"] = df["Place in 2018"].astype("str")

In [ ]:
df["Region"] = df.apply(lambda x: "Russia" if "ru" in x["Place in 2018"] else "Unknown", axis=1)

In [ ]:
df["Place in 2018"] = df["Place in 2018"].map(lambda x: "0" if x == "ru" else x)

In [ ]:
df["Place in 2012"] = df["Place in 2012"].map(lambda x: "0" if x == "nan" else x)

In [ ]:
df["Place in 2018"] = df["Place in 2018"].astype("int")
df["Place in 2012"] = df["Place in 2012"].astype("int")

In [ ]:
df["test"] = df["Place in 2012"] - df["Place in 2018"]

In [ ]:
df = df.rename(columns={"test":"Ranking change"})

In [ ]:
df["Affiliates/offices/representatives"] = df["Affiliates/offices/representatives"].fillna(0)
df["Employees"] = df["Employees"].fillna(0)

In [ ]:
df["Affiliates/offices/representatives"] = df["Affiliates/offices/representatives"].astype("int")
df["Employees"] = df["Employees"].astype("int")

In [ ]:
pd.set_option("display.float_format", lambda x: "%.0f" % x)
df["Sales volume USD bln, 2018"] = df["Sales volume USD bln, 2018"].str.replace(",", ".").astype("float") * 10 ** 9

In [ ]:
df["Sales volume USD bln, 2018"].median()

In [ ]:
px.defaults.width = 1280
px.defaults.height = 720
global_disti = px.bar(df, x="Distributor", y="Sales volume USD bln, 2018", color="Region", log_y=True, hover_data=df.columns, hover_name="Distributor", title="Global component ranking in revenue") # Global
global_disti.update_yaxes(tickprefix="$")
global_disti.add_shape(type="line", line_color="black", line_width=3, opacity=1, 
                       line_dash="dot", x0=0, x1=1, xref="paper", y0=df["Sales volume USD bln, 2018"].median(), y1=df["Sales volume USD bln, 2018"].median(), yref="y")
# global_disti.add_annotation(text="Median of the sales volume")
global_disti

In [ ]:
ru_disti = px.bar(df[df["Place in 2018"] == "RU"], x="Distributor", y="Sales volume USD bln, 2018", labels={"Distributor":"Russian Distributor"},
      title="Russian component distributor ranking in revenue", log_y=True, hover_data=df.columns, hover_name="Distributor")
ru_disti.update_yaxes(tickprefix="$")
ru_disti

In [ ]:
df["Ranking change"] = df.apply(lambda x: "-" if x["Place in 2012"] == 0 else x["Place in 2012"] - x["Place in 2018"], axis=1)

In [ ]:
df["Place in 2018"] = df["Place in 2018"].astype("str")
df["Place in 2018"] = df["Place in 2018"].map(lambda x: "RU" if x == "0" else x)
df["Place in 2012"] = df["Place in 2012"].astype("str")
df["Place in 2012"] = df["Place in 2012"].map(lambda x: "N/A" if x == "0" else x)

In [ ]:
df.to_excel(r"C:\Users\kaihsu\Desktop\global_ranking_component_distributor_1.xlsx", index=False)

In [ ]:
rank_table = df.style.format({"Sales volume USD bln, 2018":"${:,.0f}", "Employees": "{:,.0f}", "Founded":"{:.0f}"}).hide_index().set_caption("Global component distributor ranking").set_properties(**{"background-color":"black", "color": "lawngreen", "border-color":"white"})

## 輸出報告 in html

In [ ]:
rank_table

In [ ]:
df.to_excel(r"C:\Users\kaihsu\Desktop\global_ranking_component_distributor.xlsx", index=False, sheet_name="Global ranking")

In [ ]:
with open("global_component_distributor_ranking.html", "a", encoding="utf-8") as f:
    f.write(rank_table.render())
    f.write(global_disti.to_html(full_html=False, include_plotlyjs="cdn"))
#     f.write(ru_disti.to_html(full_html=False, include_plotlyjs="cdn"))
    f.close()

## 圖表美化實驗區

In [ ]:
sorted_df = df.groupby([pd.Grouper(key="預交日期", freq="M"), "BG", "Group", "單價"])[["數量", "本國幣別NTD"]].sum().reset_index()

In [ ]:
sorted_df["預交日期"] = sorted_df["預交日期"].astype("str")

In [ ]:
sorted_df = sorted_df[~sorted_df["BG"].isin(["Others"])]

In [ ]:
px.bar(sorted_df, x="預交日期", y="數量", barmode="group", color="BG", log_y=True,
      title="各BU各客戶各月出貨", text="Group").update_xaxes(title_text="Month", tickangle=45, title_font=dict(size=18), categoryorder="total descending").update_layout(
        title_font_size=26)

In [ ]:
sorted_df["預交日期"] = sorted_df["預交日期"].dt.strftime("%Y-%m")

In [ ]:
px.treemap(sorted_df, path=["預交日期", "Group"], values="本國幣別NTD", color="數量")

In [ ]:
x = px.data.gapminder()

In [ ]:
x

## 以下空白

In [ ]:
df = pd.read_clipboard()

In [ ]:
df.insert(1, 'BU', df['佳邦PN'].map(lambda x: x[0:3]))

In [ ]:
df.insert(2, 'Size', df['佳邦PN'].map(lambda x: x[3:7]))

In [ ]:
df.to_excel(r"C:\Users\kaihsu\Desktop\華科四五月forecast.xlsx", index=False)